In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip "/content/drive/MyDrive/data_large.zip" #la ubicación del archivo a descomprimir depende de cada uno

In [ ]:
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from folium.features import GeoJsonTooltip
from IPython.display import display

In [ ]:
areas_urbanas = gpd.read_file("data/Areas Urbanas/areas_urbanas.shp")
distritos_censales = gpd.read_file("data/Distritos Censales/Distritos Censales RM.shp")
centroides = gpd.read_file("data/Distritos Censales/centroides Distritos Censales RM.shp")
lineas_metro = gpd.read_file("data/Metro 2020/Lineas_2020/Lineas_2020.shp")
estaciones_metro = gpd.read_file("data/Metro 2020/Estaciones_2020/Estaciones_2020.shp")
comunas = gpd.read_file("data/Comunas/comunas.shp")

In [ ]:
areas_urbanas.columns

In [ ]:
areas_urbanas.head(3)

In [ ]:
distritos_censales.columns

In [ ]:
distritos_censales.head(2)

In [ ]:
lineas_metro.columns

In [ ]:
lineas_metro.head(2)

In [ ]:
estaciones_metro.columns

In [ ]:
estaciones_metro.head(2)

In [ ]:
areas_urbanas.crs

In [ ]:
distritos_censales.crs

In [ ]:
lineas_metro.crs

In [ ]:
estaciones_metro.crs

In [ ]:
# Transformar las capas a un CRS común (en este caso particular, EPSG 4326)
areas_urbanas = areas_urbanas.to_crs(epsg=4326)
distritos_censales = distritos_censales.to_crs(epsg=4326)
lineas_metro = lineas_metro.to_crs(epsg=4326)
estaciones_metro = estaciones_metro.to_crs(epsg=4326)
comunas = comunas.to_crs(epsg=4326)

In [ ]:
comunas_rm = comunas[comunas["Region"] == "Región Metropolitana de Santiago"]

In [ ]:
comunas_rm.head(2)

In [ ]:
mapa = folium.Map(location=[-33.4489, -70.6693], zoom_start=12)  # Coordenadas de Santiago

In [ ]:
display(mapa)

In [ ]:
folium.GeoJson(areas_urbanas, name='Áreas Urbanas', style_function=lambda feature: {
        'color': 'blue',
        'weight': 2,
        'opacity': 0.2
    }).add_to(mapa)

In [ ]:
display(mapa)

In [ ]:
colores_lineas = {
    'L1': 'red',         # Línea 1 - Rojo
    'L2': 'yellow',      # Línea 2 - Amarillo
    'L3': 'brown',       # Línea 3 - Café
    'L4': 'blue',        # Línea 4 - Azul
    'L4A': 'skyblue',    # Línea 4A - Celeste
    'L5': 'green',       # Línea 5 - Verde
    'L6': 'purple'       # Línea 6 - Morado
}

folium.GeoJson(
    lineas_metro,
    name='Líneas de Metro',
    style_function=lambda feature: {
        'color': colores_lineas.get(feature['properties']['LINEA'], 'black'),
        'weight': 4,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['LINEA'],
        aliases=['Línea: '],
        sticky=True,
    )
).add_to(mapa)

In [ ]:
display(mapa)

In [ ]:
folium.GeoJson(comunas_rm, name='Comunas',tooltip=GeoJsonTooltip(
        fields=["Comuna"],
        localize=True,
        sticky=True,
        labels=True
    )).add_to(mapa)

In [ ]:
display(mapa)

In [ ]:
estaciones_feature_group = folium.FeatureGroup(name='Estaciones de Metro')
marker_cluster = MarkerCluster().add_to(estaciones_feature_group)
for _, row in estaciones_metro.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=f"Estación: {row['NOMBRE']}",
        icon=folium.Icon(color='red', icon='train', prefix='fa')
    ).add_to(marker_cluster)
estaciones_feature_group.add_to(mapa)

In [ ]:
folium.LayerControl().add_to(mapa)
display(mapa)
mapa.save("mapa_interactivo_unificado.html")